In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from google.colab import drive

In [2]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
zip_file = '/content/gdrive/MyDrive/dataset.zip'

In [4]:
import zipfile
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('/content/gdrive/MyDrive')

In [5]:
batch_size = 64
IMAGE_SIZE = [224, 224]
data_dir_train = os.path.join('/content/gdrive/MyDrive/dataset', 'TRAIN')

train = tf.keras.utils.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="training",
  image_size= IMAGE_SIZE,
  seed=1,
  batch_size= batch_size)

val = tf.keras.utils.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="validation",
  seed=1,
  image_size= IMAGE_SIZE,
  batch_size= batch_size)

Found 3182 files belonging to 4 classes.
Using 2546 files for training.
Found 3182 files belonging to 4 classes.
Using 636 files for validation.


In [6]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
    
    layer.trainable = False
  

In [7]:

x = Flatten()(vgg.output)
prediction = Dense(4, activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
from tensorflow.keras import optimizers
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer=optimizers.Adam(learning_rate=0.01),
  metrics=['accuracy']
)

In [13]:

model.fit( 
    train,
    validation_data=val,
    epochs=20,
    batch_size=batch_size
)

Epoch 1/20
40/40 [==============================] - 19s 439ms/step - loss: 113.4356 - accuracy: 0.5958 - val_loss: 89.7483 - val_accuracy: 0.5833
Epoch 2/20
40/40 [==============================] - 19s 441ms/step - loss: 27.6637 - accuracy: 0.8213 - val_loss: 82.7489 - val_accuracy: 0.6053
Epoch 3/20
40/40 [==============================] - 19s 442ms/step - loss: 10.6823 - accuracy: 0.9128 - val_loss: 101.8335 - val_accuracy: 0.6038
Epoch 4/20
40/40 [==============================] - 19s 443ms/step - loss: 5.4562 - accuracy: 0.9438 - val_loss: 125.2008 - val_accuracy: 0.6258
Epoch 5/20
40/40 [==============================] - 19s 453ms/step - loss: 6.1031 - accuracy: 0.9497 - val_loss: 111.5959 - val_accuracy: 0.6226
Epoch 6/20
40/40 [==============================] - 20s 459ms/step - loss: 4.2344 - accuracy: 0.9576 - val_loss: 113.5481 - val_accuracy: 0.6132
Epoch 7/20
40/40 [==============================] - 20s 459ms/step - loss: 3.3090 - accuracy: 0.9698 - val_loss: 125.4942 - val_

In [15]:
data_dir_test = os.path.join('/content/gdrive/MyDrive/dataset', 'TEST')


test = tf.keras.utils.image_dataset_from_directory(
  data_dir_test,
  image_size= IMAGE_SIZE,
  seed=1,
  batch_size= batch_size)

Found 796 files belonging to 4 classes.


In [17]:
score = model.evaluate(test, verbose=False)
print('Test accuracy:', score[1])

Test accuracy: 0.6407034993171692
